**ReAct** (Reasoning + Acting) is an agent framework and prompting strategy where an LLM **thinks step-by-step** (`Thought`), **chooses an action** (`Action`), and **uses tools or APIs** (`Action Input`) to interactively solve complex tasks.

**How it works:** The LLM follows a loop of generating `Thought`, then `Action`, followed by `Observation`, repeating as needed until it produces a final `Answer`. This enables reasoning and acting in a structured, interactive way.

In [1]:
import os
from dotenv import load_dotenv

_ = load_dotenv(override=True)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

##### `LangChain Tools`

* `Wikipedia Search`

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
output = wiki.invoke({'query': 'Geoffery Hinton'})
output

'Page: Boltzmann machine\nSummary: A Boltzmann machine (also called Sherrington–Kirkpatrick model with external field or stochastic Ising model), named after Ludwig Boltzmann, is a spin-glass model with an external field, i.e., a Sherrington–Kirkpatrick model, that is a stochastic Ising model. It is a statistical physics technique applied in the context of cognitive science. It is also classified as a Markov random field.\nBoltzmann machines are theoretically intriguing because of the locality and Hebbian nature of their training algorithm (being trained by Hebb\'s rule), and because of their parallelism and the resemblance of their dynamics to simple physical processes.  Boltzmann machines with unconstrained connectivity have not been proven useful for practical problems in machine learning or inference, but if the connectivity is properly constrained, the learning can be made efficient enough to be useful for practical problems.\nThey are named after the Boltzmann distribution in sta

In [4]:
# Another example
output = wiki.invoke('Google Gemini')
print(output)

Page: Gemini (chatbot)
Summary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name, it was launched in 2023 in response to the rise of OpenAI's ChatGPT. It was previously based on the LaMDA and PaLM LLMs.
LaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in a limited capacity in March 2023 before expanding to other countries in May. Bard took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. In February 2024, Bard and Duet AI, another artificial intelligence product from Google, were unified under the Gemini brand, coinciding with the launch of 

* `DuckDuckGo Search`
> DuckDuckGo is a privacy-focused search engine that doesn't track users' searches

In [5]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults


In [6]:
search = DuckDuckGoSearchRun()

output = search.invoke("Where was Geoffery Hinton born ?")
output

'Geoffrey Hinton (born December 6, 1947, London, England) is a British-Canadian cognitive psychologist and computer scientist known as the "godfather of AI." He revolutionized the field of artificial intelligence with his work on neural network models. He contributed significantly to AI research with novel insights and key discoveries in the areas of backpropagation, Boltzmann machines ... Geoffrey Hinton\'s journey into the world of AI began with a rich intellectual heritage and a diverse educational background. Hinton was born on December 6, 1947, in Wimbledon, London, England. His family included notable intellectuals like mathematician Mary Everest Boole and logician George Boole. His father, James Hinton, was a surgeon. Geoffrey Everest Hinton is a British-Canadian computer scientist and cognitive psychologist, most noted for his work on artificial neural networks. ... Geoffrey Hinton was born on December 6, 1947, in Wimbledon, England, United Kingdom. His birth geographical coord

In [7]:
search.name, search.description

('duckduckgo_search',
 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [8]:
wrapper = DuckDuckGoSearchAPIWrapper(region="xa-ar" , max_results=3 , safesearch="moderate")
search = DuckDuckGoSearchResults(api_wrapper=wrapper , source = "news")
output = search.run("Egypt")
output

"snippet: Learn about Egypt, a country in Africa and Asia with a rich and ancient civilisation. Explore its geography, culture, economy, government, foreign relations and more on Wikipedia., title: Egypt - Wikipedia, link: https://en.wikipedia.org/wiki/Egypt, snippet: A comprehensive overview of Egypt's geography, people, society, government, economy, energy, communications, transportation, military, and transnational issues. Compare Egypt with other countries by various indicators and rankings., title: Egypt - The World Factbook, link: https://www.cia.gov/the-world-factbook/countries/egypt/, snippet: The website of the Presidency of the Arab Republic of Egypt provides news, speeches, and initiatives of President Abdel Fattah El-Sisi. It also features the Museum of the Revolutionary Command Council and the President's answers to questions., title: Presidency of the Arab Republic of Egypt Website, link: https://www.presidency.eg/EN, snippet: ابقَ على اطلاعٍ مع أحدث التحديثات حول مصر من 

In [9]:
import re

# Adjusted pattern to handle multiline and ensure proper capturing
pattern = r'snippet: (.*?),\s*title: (.*?),\s*link: (https?://[^\s,]+)'

# Find all matches in the output text
matches = re.findall(pattern, output, re.DOTALL)

# Print each match in the desired format
for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Learn about Egypt, a country in Africa and Asia with a rich and ancient civilisation. Explore its geography, culture, economy, government, foreign relations and more on Wikipedia.
Title: Egypt - Wikipedia
Link: https://en.wikipedia.org/wiki/Egypt

--------------------------------------------------
Snippet: A comprehensive overview of Egypt's geography, people, society, government, economy, energy, communications, transportation, military, and transnational issues. Compare Egypt with other countries by various indicators and rankings.
Title: Egypt - The World Factbook
Link: https://www.cia.gov/the-world-factbook/countries/egypt/

--------------------------------------------------
Snippet: The website of the Presidency of the Arab Republic of Egypt provides news, speeches, and initiatives of President Abdel Fattah El-Sisi. It also features the Museum of the Revolutionary Command Council and the President's answers to questions.
Title: Presidency of the Arab Republic of Egypt Web

* `ReAct Agent`
> ReAct is an approach that combines reasoning (thought process) with action-taking capabilities in AI agents. In the ReAct framework, agents can generate outputs by reasoning about the task at hand and taking actions iteratively, such as retrieving information, answering questions, or interacting with tools. This allows the agent to handle complex tasks more effectively by breaking down problems and solving them step-by-step, mimicking human-like reasoning

In [13]:
from langchain.prompts import PromptTemplate
from langchain.agents import Tool , AgentExecutor , create_react_agent
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools.python.tool import PythonREPLTool


In [18]:
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key=GOOGLE_API_KEY)

# Define a template for answering questions
template = '''
Answer the following questions in ARABIC as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = hub.pull('hwchase17/react')
prompt

d:\AAAAAAA\RAG\Projects\LangChain - Basics, Chat and RAG\LangChain\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [19]:
# Create tools for the agent
# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=20
)

In [20]:
question = 'Generate the first 10 numbers in the Fibonacci series.'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})
output



> Entering new AgentExecutor chain...
Thought:I need to generate the first 10 Fibonacci numbers and then translate them into Arabic.  I'll use Python to generate the numbers and then manually translate them.

Action: Python REPL

Action Input:
```python
def fibonacci(n):
    a, b = 0, 1
    fib_sequence = []
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

print(fibonacci(10))
```[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Thought:Now I need to translate the numbers [0, 1, 1, 2, 3, 5, 8, 13, 21, 34] into Arabic.

Action: None.  Manual translation.Invalid Format: Missing 'Action Input:' after 'Action:'Thought:I need to generate the first 10 Fibonacci numbers and then translate them into Arabic.  I'll use Python to generate the numbers and then manually translate them.

Action: Python REPL

Action Input:
```python
def fibonacci(n):
    a, b = 0, 1
    fib_sequence = []
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a 

{'input': '\nAnswer the following questions in ARABIC as best you can.\nQuestions: Generate the first 10 numbers in the Fibonacci series.\n',
 'output': 'صفر، واحد، واحد، اثنان، ثلاثة، خمسة، ثمانية، ثلاثة عشر، واحد وعشرون، أربعة وثلاثون'}

In [22]:
question = 'Who is the current prime minister of the Egypt?'

output = agent_executor({
    "input" : prompt_template.format(q = question)
})

output

C:\Users\user\AppData\Local\Temp\ipykernel_17444\2908821828.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = agent_executor({




> Entering new AgentExecutor chain...
Thought: I need to find the current Prime Minister of Egypt.  I'll use DuckDuckGo Search to get the most up-to-date information, then translate the answer to Arabic.

Action: DuckDuckGo Search
Action Input: "current prime minister of Egypt"This is a list of current heads of state and heads of government. In some cases, ... Egypt: President - Abdel Fattah el-Sisi: Prime Minister - Mostafa Madbouly ... Prime Minister - Mahn Winn Khaing Thann: Hamas government in the Gaza Strip Egypt's presidency said in a statement on Monday that Madbouly, who has been prime minister since 2018, has been tasked with shaping a new and more capable government to tackle issues at home ... Egypt's Cabinet of Ministers announced its new ministerial lineup on Wednesday, July 3rd, one month following the resignation of Prime Minister Mostafa Madbouly's government. Madbouly will continue to lead the new government, a role he has held since 2018. As part of the new minister

{'input': '\nAnswer the following questions in ARABIC as best you can.\nQuestions: Who is the current prime minister of the Egypt?\n',
 'output': 'مصطفى مدبولي'}

In [23]:
import time
time.sleep(5)

question = 'Tell me about Mohamed ALi Basha early life'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})
output



> Entering new AgentExecutor chain...
Thought:I need to find information about Mohamed Ali Pasha's early life.  A Wikipedia search seems like the best approach to get a comprehensive overview, and then I can translate the relevant information into Arabic.

Action: Wikipedia
Action Input: Mohamed Ali Pasha

d:\AAAAAAA\RAG\Projects\LangChain - Basics, Chat and RAG\LangChain\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\AAAAAAA\RAG\Projects\LangChain - Basics, Chat and RAG\LangChain\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Muhammad Ali of Egypt
Summary: Muhammad Ali (4 March 1769 – 2 August 1849) was the Ottoman Albanian viceroy and governor who became the de facto ruler of Egypt from 1805 to 1848, widely considered the founder of modern Egypt. At the height of his rule, he controlled Egypt, Sudan, Hejaz, the Levant, Crete and parts of Greece.
Born in Kavala in the Rumelia Eyalet, Ali was a military commander in an Albanian Ottoman force sent to recover Egypt from French occupation following Napoleon's withdrawal. He rose to power through a series of political maneuvers, and in 1805 he was named Wāli (governor) of Egypt and gained the rank of Pasha. As Wāli, Ali attempted to modernize Egypt by instituting dramatic reforms in the military, economic and cultural spheres. He also initiated a violent purge of the Mamluks, consolidating his rule and permanently ending the Mamluk hold over Egypt.
Militarily, Ali recaptured the Arabian territories for the sultan, and conquered Sudan of his own accord. His

{'input': '\nAnswer the following questions in ARABIC as best you can.\nQuestions: Tell me about Mohamed ALi Basha early life\n',
 'output': 'ولد محمد علي باشا في 4 مارس 1769 في كافالا، وهي مدينة ساحلية تقع الآن في اليونان، ولكنها كانت آنذاك جزءًا من الإمبراطورية العثمانية. كان والده ألبانيًا. وكان قائدًا عسكريًا في قوة عثمانية ألبانية أُرسلت لاستعادة مصر من الاحتلال الفرنسي بعد انسحاب نابليون.  وصعد إلى السلطة من خلال سلسلة من المناورات السياسية.'}